# Training Visualization

In this part we’ll see how to create a simple but wrong model with Keras, and, gradually, how it can be improved with step-by-step debugging and understanding with TensorBoard.

*Please, download [log files](https://drive.google.com/open?id=0B6VSPXOeu5J0TnRhS18xS2pBMFU) first.*

Let's start from importing all necessary components/layers for the future CNN.

In [9]:
from keras.models import Model
from keras.layers import Convolution2D, BatchNormalization, MaxPooling2D, Flatten, Dense
from keras.layers import Input, Dropout
from keras.layers.advanced_activations import ELU
from keras.regularizers import l2
from keras.optimizers import SGD

import tensorflow as tf

from settings import *
# from somethere import train_generator, val_generator()

## Toy convolutional model for classification
First, we create a skeleton for a model with one convolutional and one dense layer

In [10]:
def build_model(n_classes):

    input_shape = (N_MEL_BANDS, SEGMENT_DUR, 1)
    channel_axis = 3
    melgram_input = Input(shape=input_shape)

    m_size = 70
    n_size = 3
    n_filters = 64
    maxpool_const = 4

    x = Convolution2D(n_filters, (m_size, n_size),
                      padding='same',
                      kernel_initializer='zeros',
                      kernel_regularizer=l2(1e-5))(melgram_input)

    x = BatchNormalization(axis=channel_axis)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(N_MEL_BANDS, SEGMENT_DUR/maxpool_const))(x)
    x = Flatten()(x)

    x = Dropout(0.5)(x)
    x = Dense(n_classes, kernel_initializer='zeros', kernel_regularizer=l2(1e-5), 
              activation='softmax', name='prediction')(x)

    model = Model(melgram_input, x)

    return model

model = build_model(IRMAS_N_CLASSES)

We can train the model on IRMAS data using the training procedure below.

First, we have to define the optimizer. We're using Stochastic Gradient Descent with Momentum

In [12]:
init_lr = 0.001
optimizer = SGD(lr=init_lr, momentum=0.9, nesterov=True)

Now we can check the model structure, specify which metrics we would like to keep eye on and compile the model.

In [13]:
model.summary()
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 96, 128, 1)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 96, 128, 64)       13504     
_________________________________________________________________
batch_normalization_3 (Batch (None, 96, 128, 64)       256       
_________________________________________________________________
elu_3 (ELU)                  (None, 96, 128, 64)       0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1, 4, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
__________

From the previous part, we have two generators which can provide us training samples and validation samples. 
We will use them during the training. We also specify the number of steps per epoch, the total number of epoch and the log verbosity level 

In [ ]:
model.fit_generator(train_generator(),
                    steps_per_epoch=STEPS_PER_EPOCH,
                    nb_epoch=MAX_EPOCH_NUM,
                    verbose=2,
                    validation_data=val_generator(),
                    class_weight=None,
                    nb_worker=1)

As we can see, neither validation nor the training metrics has improved, so we need to explore that's wrong with the model. Keras Callbacks will help us in this.

## Keras Callbacks

The Callback in Keras is a set of functions to be applied on a certain event during the training process.
The typical triggers for events are:
* on_epoch_begin
* on_epoch_end
* on_batch_begin
* on_batch_end
* on_train_begin
* on_train_end

There are some useful callbacks:

In [15]:
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, TensorBoard

early_stopping = EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING_EPOCH)
save_clb = ModelCheckpoint("{weights_basepath}/".format(weights_basepath=MODEL_WEIGHT_BASEPATH) +
                           "epoch.{epoch:02d}-val_loss.{val_loss:.3f}",
                           monitor='val_loss',
                           save_best_only=True)

Let's get acquainted with the TensorBoard Callback.

The parameters are:
* log_dir - where to store the logs, metadata, and events of the model training process
* write_graph - whether or not to write the graph of data and control dependencies
* write_grads - whether or not to save the parameters of the model for visualisation
* histogram_freq - how often to save the parameters of the model
* write_images - whether or not to save the weight and visualise them as images

Additional parameters includes embeddings visualisation but, as we will see, it's not suitable for our case.

In [ ]:
tb = TensorBoard(log_dir='./example_1',
                 write_graph=True, write_grads=True, 
                 write_images=True, histogram_freq=1)
# if we want to compute activations and weight histogram, we need to specify the validation data for that. 
tb.validation_data = val_generator()

Now we can add the callbacks to the training process and observe the corresponding enevts and obtain the corresponding logs. 

In [ ]:
model.fit_generator(train_generator(),
                    steps_per_epoch=STEPS_PER_EPOCH,
                    nb_epoch=MAX_EPOCH_NUM,
                    verbose=2,
                    validation_data=val_generator(),
                    callbacks=[save_clb, early_stopping, tb],
                    class_weight=None,
                    nb_worker=1)

You can download the event files for all runs from [here](https://drive.google.com/open?id=0B6VSPXOeu5J0TnRhS18xS2pBMFU).

Now create the `./logs` directory and launch TensorBoard

``` bash
tar -xvzf logs.tar.gz
cd logs
tensorboard --logdir ./example_1
```

and navigate to http://0.0.0.0:6006

We can notice, that it's almoust impossible to see anything on the Graphs tab but we can see vividly that the metrics on the Scalar tab are not improving and the gradients values on the Histograms tab are zero.

Our problem is in the weights initialization `kernel_initializer='zeros'` so now we can fix it and define new model.

In [18]:
def build_model(n_classes):

    input_shape = (N_MEL_BANDS, SEGMENT_DUR, 1)
    channel_axis = 3
    melgram_input = Input(shape=input_shape)

    m_size = 70
    n_size = 3
    n_filters = 64
    maxpool_const = 4

    x = Convolution2D(n_filters, (m_size, n_size),
                      padding='same',
                      kernel_initializer='he_normal',
                      kernel_regularizer=l2(1e-5))(melgram_input)

    x = BatchNormalization(axis=channel_axis)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(N_MEL_BANDS, SEGMENT_DUR/maxpool_const))(x)
    x = Flatten()(x)

    x = Dropout(0.5)(x)
    x = Dense(n_classes, kernel_initializer='he_normal', kernel_regularizer=l2(1e-5), 
              activation='softmax', name='prediction')(x)

    model = Model(melgram_input, x)

    return model

model = build_model(IRMAS_N_CLASSES)

If you will repeate the training process, you may notice that classification performance improved significantly.

Have a look for a new log file in the `./example_2` directory and restart TensorBoard to explore new data.

``` bash
cd logs
tensorboard --logdir ./example_2
```

## TensorFlow name scopes

You might have noticed the hell on the Graphs tab.
That's because TensorBoard can't connect all the data nodes in the model and operations in the training process together, it's smart enought to group the nodes with similar structure but don't expect too much.

In order to make the better graph visualisation, we need to define the name scopes for each logical layer and each operation we want to see as an individual element.

We can do it just by adding `with tf.name_scope(name_scope)` clause:

In [ ]:
global_namescope = 'train'

def build_model(n_classes):

    with tf.name_scope('input'):
        input_shape = (N_MEL_BANDS, SEGMENT_DUR, 1)
        channel_axis = 3
        melgram_input = Input(shape=input_shape)

        m_size = [5, 5]
        n_size = [5, 5]
        n_filters = 64
        maxpool_const = 8

    with tf.name_scope('conv1'):
        x = Convolution2D(n_filters, (m_size[0], n_size[0]),
                          padding='same',
                          kernel_initializer='he_uniform')(melgram_input)
        x = BatchNormalization(axis=channel_axis)(x)
        x = ELU()(x)
        x = MaxPooling2D(pool_size=(maxpool_const, maxpool_const))(x)

    with tf.name_scope('conv2'):
        x = Convolution2D(n_filters*2, (m_size[1], n_size[1]),
                          padding='same',
                          kernel_initializer='he_uniform')(x)
        x = BatchNormalization(axis=channel_axis)(x)
        x = ELU()(x)
        x = MaxPooling2D(pool_size=(maxpool_const, maxpool_const))(x)
        x = Flatten()(x)

    with tf.name_scope('dense1'):
        x = Dropout(0.5)(x)
        x = Dense(n_filters, kernel_initializer='he_uniform', name='hidden')(x)
        x = ELU()(x)

    with tf.name_scope('dense2'):
        x = Dropout(0.5)(x)
        x = Dense(n_classes, kernel_initializer='he_uniform', activation='softmax', name='prediction')(x)

    model = Model(melgram_input, x)

    return model

model = build_model(IRMAS_N_CLASSES)

with tf.name_scope('optimizer'):
    optimizer = SGD(lr=init_lr, momentum=0.9, nesterov=True)

with tf.name_scope('model'):
    model = build_model(IRMAS_N_CLASSES)

# for the sake of memory, only graphs now
with tf.name_scope('callbacks'):
    # The TensorBoard developers are strongly encourage us to use different directories for every run
    tb = TensorBoard(log_dir='./example_3', write_graph=True)

# yes, we need to recompile the model every time
with tf.name_scope('compile'):
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# and preudo-train the model
with tf.name_scope(global_namescope):
    model.fit_generator(train_generator(),
                        steps_per_epoch=STEPS_PER_EPOCH,
                        nb_epoch=1, # one epoch to save the graphs
                        verbose=2,
                        validation_data=val_generator(),
                        callbacks=[tb],
                        nb_worker=1)

Have a look for a new log file in the `./example_3` directory and restart TensorBoard to explore new data.

``` bash
cd logs
tensorboard --logdir ./example_3
```

## Embeddings Visualisation

With TensorBoard we can also visualise the embeddings of the model.
Unfortunately, the present implementation in Keras doesn't allow to store the true embeddings on the data (it only saves weights for the layers _or_ embeddings for the Embedding layers, that is siutable for word embeddings like word2vec, but is not suitable in our case). 

However, we can write our custom callback and use it to store the embeddings on validation data during the training process. We will follow the notation from TensorBoard callback, but add some functionality:

* layer_names - a list of names of layers to keep eye on
* metadata - a path to a TSV file with associated meta information (labels, notes, etc.), [format and details](https://www.tensorflow.org/get_started/embedding_viz) 
* sprite - a path to a sprite image, [format and details](https://www.tensorflow.org/get_started/embedding_viz) 
* sprite_shape - a list with values [M, N], the dimentionality of a single image, [format and details](https://www.tensorflow.org/get_started/embedding_viz)

In [ ]:
class TensorBoardEmbeddings(Callback):
    """Tensorboard Embeddings visualization callback."""

    def __init__(self, log_dir='./logs_embed',
                 batch_size=32,
                 freq=0,
                 layer_names=None,
                 metadata=None,
                 sprite=None,
                 sprite_shape=None):
        super(TensorEmbeddings, self).__init__()
        self.log_dir = log_dir
        self.freq = freq
        self.layer_names = layer_names
        # Notice, that only one file is supported in the present callback
        self.metadata = metadata
        self.sprite = sprite
        self.sprite_shape = sprite_shape
        self.batch_size = batch_size

    def set_model(self, model):
        self.model = model
        self.sess = K.get_session()
        self.summary_writer = tf.summary.FileWriter(self.log_dir)
        self.embeddings_ckpt_path = os.path.join(self.log_dir, 'keras_embedding.ckpt')

        if self.freq and self.validation_data:
            # define tensors to compute embeddings on
            embeddings_layers = [layer for layer in self.model.layers
                                 if layer.name in self.layer_names]
            self.output_tensors = [tf.get_default_graph().get_tensor_by_name(layer.get_output_at(0).name)
                                   for layer in embeddings_layers]

            # create configuration for embeddings visualisation
            config = projector.ProjectorConfig()
            for i in range(len(self.output_tensors)):
                embedding = config.embeddings.add()
                embedding.tensor_name = '{ns}/embedding_{i}'.format(ns=global_namescope, i=i)

                # Simpliest metadata handler, a single file for all embeddings
                if self.metadata:
                    embedding.metadata_path = self.metadata

                # Sprite image handler
                if self.sprite and self.sprite_shape:
                    embedding.sprite.image_path = self.sprite
                    embedding.sprite.single_image_dim.extend(self.sprite_shape)

            # define TF variables to store the embeddings during the training
            # Notice, that only 1D embeddings are supported
            self.embedding_vars = [tf.Variable(np.zeros((len(self.validation_data[0]),
                                                         self.output_tensors[i].shape[1]),
                                                        dtype='float32'),
                                               name='embedding_{}'.format(i))
                                   for i in range(len(self.output_tensors))]
            # add TF variables into computational graph
            for embedding_var in self.embedding_vars:
                self.sess.run(embedding_var.initializer)

            # save the config and setup TF saver for embedding variables
            projector.visualize_embeddings(self.summary_writer, config)
            self.saver = tf.train.Saver(self.embedding_vars)

    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data and self.freq:
            if epoch % self.freq == 0:

                val_data = self.validation_data
                tensors = (self.model.inputs +
                           self.model.targets +
                           self.model.sample_weights)
                all_embeddings = [[]]*len(self.output_tensors)

                if self.model.uses_learning_phase:
                    tensors += [K.learning_phase()]

                assert len(val_data) == len(tensors)
                val_size = val_data[0].shape[0]
                i = 0
                # compute embeddings batch by batch on validation data
                while i < val_size:
                    step = min(self.batch_size, val_size - i)
                    batch_val = []
                    batch_val.append(val_data[0][i:i + step])
                    batch_val.append(val_data[1][i:i + step])
                    batch_val.append(val_data[2][i:i + step])
                    if self.model.uses_learning_phase:
                        batch_val.append(val_data[3])
                    feed_dict = dict(zip(tensors, batch_val))
                    tensor_outputs = self.sess.run(self.output_tensors, feed_dict=feed_dict)
                    for output_idx, tensor_output in enumerate(tensor_outputs):
                        all_embeddings[output_idx].extend(tensor_output)
                    i += self.batch_size
                
                # rewrite the current state of embeddings with new values
                for embedding_idx, embed in enumerate(self.embedding_vars):
                    embed.assign(np.array(all_embeddings[embedding_idx])).eval(session=self.sess)
                # save embedding variables (the saver is defied above, so it knows what to store)
                self.saver.save(self.sess, self.embeddings_ckpt_path, epoch)

        self.summary_writer.flush()

    def on_train_end(self, _):
        self.summary_writer.close()

Now we can add the new callback, recompile and retrain the model.

In [ ]:
embeddings_to_monitor = ['hidden']
# find the files precomputed in ./logs_embed directory 
metadata_file_name = 'metadata.tsv'
sprite_file_name = 'sprite.png'
sprite_shape = [N_MEL_BANDS, SEGMENT_DUR]

with tf.name_scope('callbacks'):
    tbe = TensorBoardEmbeddings(log_dir='./logs_embed', freq=1,
                           layer_names=embeddings_to_monitor,
                           metadata=metadata_file_name,
                           sprite=sprite_file_name,
                           sprite_shape=sprite_shape)
    tbe.validation_data = val_generator()

with tf.name_scope('compile'):
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

with tf.name_scope(global_namescope):
    model.fit_generator(train_generator(),
                        steps_per_epoch=STEPS_PER_EPOCH,
                        nb_epoch=MAX_EPOCH_NUM,
                        verbose=2,
                        callbacks=[tbe],
                        validation_data=val_generator(),
                        class_weight=None,
                        nb_worker=1)

For the sake of time, we're going to skip the trainig process. You can find the corresponding data in `./logs_embed` directory.

Restart TensorBoard, navigate to http://0.0.0.0:6006/ and now we can discuss the embeddings in details.
``` bash
cd logs
tensorboard --logdir ./logs_embed
```